같은 질문으로 서로 다른 모델의 답변 비교해보기

In [1]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, pipeline
from peft import PeftModel, PeftConfig, prepare_model_for_kbit_training
from transformers import pipeline



def load_model(model_id):
    
    print(model_id)
    
    bnb_config = BitsAndBytesConfig(    # 모델의 성능을 유지하면서 메모리 사용을 최적화하고, 하드웨어 환경에 맞게 데이터를 처리하는 데 도움
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    # Set the available GPU devices
    # available_gpus = [0]    # GPU의 개수가 1개 일때
    available_gpus = [0, 1]  # GPU의 개수가 2개 일때
    # available_gpus = [0,1,2,3]    # GPU의 개수가 4개 일때

    # Function to find the GPU with the least memory usage
    def get_least_memory_gpu():
        least_memory = float('inf')
        least_memory_gpu = None
        for gpu in available_gpus:
            allocated_memory = torch.cuda.memory_allocated(gpu)
            if allocated_memory < least_memory:
                least_memory = allocated_memory
                least_memory_gpu = gpu
        return least_memory_gpu

    # Allocate the remaining GPU memory to the current process
    device = get_least_memory_gpu()
    torch.cuda.set_device(device)

    # Load the model and allocate GPU memory
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device)


    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # 모델의 훈련 과정에서 필요한 메모리 양을 줄일 수 있습니다.
    if model_id !=  'beomi/KoAlpaca-KoRWKV-6B':
        model.gradient_checkpointing_enable()           # Activates gradient checkpointing for the current model.
        model = prepare_model_for_kbit_training(model)
            
    model.eval()
    model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
    
    return model, tokenizer

def prompt_formating(model_id,user_input):
    
    # llama-2의 프롬프트 형식
    """<s>[INST] <<SYS>>
    {{ system_prompt }}
    <</SYS>>

    {{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>\
    <s>[INST] {{ user_msg_2 }} [/INST] {{ model_answer_2 }} </s>\
    <s>[INST] {{ user_msg_3 }} [/INST]"""
    
    # DEFAULT_SYSTEM_PROMPT 
    # You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
    # If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

    
    
    if model_id == "beomi/llama-2-ko-7b" or model_id == 'beomi/KoAlpaca-KoRWKV-6B' or model_id == 'kfkas/Llama-2-ko-7b-Chat' :     # beomi FORMAT
        prompt = f"### 질문: {user_input}\n\n### 답변:"         
        
    elif model_id == 'nlpai-lab/kullm-polyglot-5.8b-v2':        # kullm-polyglot-5.8b-v2 FORMAT
        prompt = f"""프롬프트 확인 : 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
{user_input}

### 입력:


### 응답:"""
        
        
    else:       # 기본 FORMAT
        prompt = f"### instruction: {user_input}\n\n### Response:\n"            
        # prompt = f'### 질문: {user_input}\n\n### 답변:'      # beomi/KoAlpaca-KoRWKV-6B and "beomi/polyglot-ko-12.8b-safetensors FORMAT        
    
    return prompt
    
def gen(model, model_id, tokenizer, user_input, max_new_tokens=256):     # gen 의 형식은 모델마다 다를 수 있으니 참고하여 프롬프트를 설정해주세요.

    if model_id ==  'beomi/KoAlpaca-KoRWKV-6B':
        eos_token_id = 0
    else:
        eos_token_id = 2
    
    prompt = prompt_formating(model_id,user_input)
        
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=max_new_tokens,
        early_stopping=True,
        do_sample=False,                 # False : grid 샘플링
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id
    )
    return tokenizer.decode(gened[0])

    
# def generate_response(model, tokenizer, input_text):
#     inputs = tokenizer.encode(input_text, return_tensors="pt")
#     outputs = model.generate(inputs, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return response




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


In [2]:
"""
MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join(f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        print(f'프롬프트 확인 : {res}')
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [6]:
MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=1, non_blocking=True)
model.eval()

pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=1)

prompter = Prompter("kullm")


def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

주석 풀면서 load하고 싶은 모델 load 하기

In [2]:
# KoRWKV, KoRWKV_tokenizer = load_model('beomi/KoAlpaca-KoRWKV-6B')
# Llama_2_ko_7b, Llama_2_ko_7b_tokenizer = load_model("beomi/llama-2-ko-7b")
# Llama_2_ko_7b_Chat, Llama_2_ko_7b_Chat_tokenizer = load_model('kfkas/Llama-2-ko-7b-Chat')
# polyglot_ko, polyglot_ko_tokenizer = load_model("EleutherAI/polyglot-ko-5.8b")
polyglot_ko_kullm, polyglot_ko_kullm_tokenizer = load_model('nlpai-lab/kullm-polyglot-5.8b-v2')

# KoRWKV_config = {
#     'model' : KoRWKV,
#     'tokenizer' : KoRWKV_tokenizer,
#     'model_id' : 'beomi/KoAlpaca-KoRWKV-6B'
# }

# Llama_2_ko_7b_config = {
#     'model' : Llama_2_ko_7b,
#     'tokenizer' : Llama_2_ko_7b_tokenizer,
#     'model_id' : "beomi/llama-2-ko-7b"
# }

# Llama_2_ko_7b_Chat_config = {
#     'model' : Llama_2_ko_7b_Chat,
#     'tokenizer' : Llama_2_ko_7b_Chat_tokenizer,
#     'model_id' : 'kfkas/Llama-2-ko-7b-Chat'
# }


# polyglot_ko_config = {
#     'model' : polyglot_ko,
#     'tokenizer' : polyglot_ko_tokenizer,
#     'model_id' : "EleutherAI/polyglot-ko-5.8b"
# }


polyglot_ko_kullm_config = {
    'model' : polyglot_ko_kullm,
    'tokenizer' : polyglot_ko_kullm_tokenizer,
    'model_id' : 'nlpai-lab/kullm-polyglot-5.8b-v2'
}


nlpai-lab/kullm-polyglot-5.8b-v2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
"""MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'"""


import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

KoAlpaca_pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)
def ask(x, context='', is_input_full=False):
    ans = KoAlpaca_pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.9,
        return_full_text=True,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [7]:
ask("캠핑 여행에 필요한 10가지 품목의 목록")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 캠핑 여행에 필요한 10가지 품목의 목록

### 답변:1. 천막
                    
2. 타프
                    
3. 테이블
                    
4. 체어
                    
5. 부츠
                    
6. 바인딩
                    
7. 스페츠
                    
8. 플렉스
                    
9.angle
                    
10.보조배터리
                    

이외에도, 랜턴, 급식용 테이블, 철모, 여분의 의류 등이 필요합니다.


위에서 주석 풀었던 모델들을 마찬가지로 아래 코드에서도 주석 풀어주기

In [4]:
responses = []

model_ids = [polyglot_ko_kullm_config['model_id']]

user_input = '캠핑 여행에 필요한 10가지 품목의 목록'   # 캠핑 여행에 필요한 10가지 품목의 목록을 생성합니다. /kfkas/Llama-2-ko-7b-Chat

if user_input:
    # KoRWKV_6B_answer = KoRWKV_gen(user_input)
    # KoAlpaca_answer = ask(user_input)
    # Llama_2_ko_7b_answer = gen(model=Llama_2_ko_7b_config['model'], model_id=Llama_2_ko_7b_config['model_id'], tokenizer=Llama_2_ko_7b_config['tokenizer'], user_input=user_input)
    # Llama_2_ko_7b_Chat_answer = gen(model=Llama_2_ko_7b_Chat_config['model'], model_id=Llama_2_ko_7b_Chat_config['model_id'], tokenizer=Llama_2_ko_7b_Chat_config['tokenizer'], user_input=user_input)
    # polyglot_ko_answer = gen(model=polyglot_ko_config['model'], model_id=polyglot_ko_config['model_id'], tokenizer=polyglot_ko_config['tokenizer'], user_input=user_input)
    polyglot_ko_kullm_answer = gen(model=polyglot_ko_kullm_config['model'], model_id=polyglot_ko_kullm_config['model_id'], tokenizer=polyglot_ko_kullm_config['tokenizer'], user_input=user_input)
    # polyglot_ko_kullm_answer_2 = infer(input_text=user_input)
    
    # responses.append(KoAlpaca_answer)
    # responses.append(Llama_2_ko_7b_answer)
    # responses.append(Llama_2_ko_7b_Chat_answer)
    # responses.append(polyglot_ko_answer)
    responses.append(polyglot_ko_kullm_answer)
    # responses.append(polyglot_ko_kullm_answer_2)
    
        
    # 답변들을 출력
    for id, response in zip(model_ids,responses):
        print()
        print(f'< {id} >')
        print(f"{response}")
        print('------')


< nlpai-lab/kullm-polyglot-5.8b-v2 >
프롬프트 확인 : 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
캠핑 여행에 필요한 10가지 품목의 목록

### 입력:


### 응답:
캠핑 여행에 필요한 10가지 품목은 다음과 같습니다:

1. 텐트: 텐트는 캠핑 여행의 필수 요소입니다. 텐트는 비바람을 막아주고 편안한 잠자리를 제공하며, 캠핑장에서의 하룻밤을 위한 아늑한 공간을 제공합니다.

2. 침낭: 침낭은 따뜻하고 편안한 잠자리를 제공하는 필수 아이템입니다. 캠핑장에서는 담요나 담요를 제공하지 않는 경우가 많으므로 침낭이 필요합니다.

3. 캠핑 의자: 캠핑 의자에 앉아 휴식을 취하거나 식사를 하는 것은 캠핑 여행의 또 다른 즐거움입니다. 캠핑 의자가 없으면 캠핑장에서 식사를 하거나 휴식을 취하기가 어려울 수 있습니다.

4. 스토브: 스토브는 캠핑 여행의 필수품입니다. 스토브는 요리를 하고, 난방을 하고, 불을 피우는 데 사용할 수 있습니다.

5. 조리 도구: 캠핑 여행에는 요리를 하는 데 필요한 도구가 필요합니다. 캠핑용 칼, 도마, 식기 등이 포함됩니다.

6. 조리 도구: 캠핑 여행에는 요리를 하는 데 필요한 도구가 필요합니다. 캠핑용 칼,
------


모든 모델에서 학습한 내용에 대해서는 답변이 준수하게 나옴.
그 중 가장 준수한 모델은 구름 모델.<br>

괜찮은 모델 : 
1. **beomi/KoAlpaca-Polyglot-5.8B** Polyglot 모델을 KoAlpaca Datasets(21k)로 학습한 모델 <br>
2. **nlpai-lab/kullm-polyglot-5.8b-v2** Polyglot 모델을 **GPT4ALL, Dolly, Vicuna 데이터셋(153k, nlpai-lab/kullm-v2)으로 학습한 모델, 8에폭 진행,  Low Rank Adaptation (LoRA) <br>

개인적인 성능 평가 결과 : **beomi/KoAlpaca-Polyglot-5.8B < nlpai-lab/kullm-polyglot-5.8b-v2**   (둘다 성능은 괜찮으나 nlpai-lab에서 만든 모델이 좀더 좋다.)